In [ ]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
from tqdm import tqdm
import io
import os
from global_variables import *

poi_csv_path = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_poi_with_store_id.csv.gz'
poi = pd.read_csv(poi_csv_path)
output_path_parks = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_parks.csv.gz'

output_path_hospitals = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_hospitals.csv.gz'

output_path_pharmacies = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_pharmacies.csv.gz'
parks = poi[poi['SUB_CATEGORY'] == "Nature Parks and Other Similar Institutions"]

with gzip.open(output_path_parks, 'wt', encoding='utf-8') as gzipped_file:
    parks.to_csv(gzipped_file, index=False)
hospitals = poi[poi['SUB_CATEGORY'].isin(
    ['Offices of Physicians (except Mental Health Specialists)', 'General Medical and Surgical Hospitals',
     'Specialty (except Psychiatric and Substance Abuse) Hospitals', 'Outpatient Care Centers']) | poi[
                    'TOP_CATEGORY'].isin(['Outpatient Care Centers'])]

with gzip.open(output_path_hospitals, 'wt', encoding='utf-8') as gzipped_file:
    hospitals.to_csv(gzipped_file, index=False)
pharmacies = poi[poi['SUB_CATEGORY'].isin(['Pharmacies and Drug Stores'])]

with gzip.open(output_path_pharmacies, 'wt', encoding='utf-8') as gzipped_file:
    pharmacies.to_csv(gzipped_file, index=False)
parks = pd.read_csv(output_path_parks)
hospitals = pd.read_csv(output_path_hospitals)
pharmacies = pd.read_csv(output_path_pharmacies)
date_start = '2023-01-01'
date_end = '2024-01-31'

file_suffix = date_start.replace('-', '') + "_" + date_end.replace('-', '')
file_suffix
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_ft_data' + file_suffix + '.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
                           params={'page': page,
                                   'partition_key_after': date_start,  # optionally set date value here
                                   'partition_key_before': date_end},  # optionally set date value here
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                    })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")

        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)

        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['PLACEKEY'].isin(parks['PLACEKEY']) |
                         df['PLACEKEY'].isin(hospitals['PLACEKEY']) |
                         df['PLACEKEY'].isin(pharmacies['PLACEKEY'])]

        selected_columns = ['PLACEKEY', 'LOCATION_NAME', 'STREET_ADDRESS', 'POSTAL_CODE', 'REGION', 'DATE_RANGE_START',
                            'DATE_RANGE_END', 'RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'VISITS_BY_DAY',
                            'DISTANCE_FROM_HOME', 'MEDIAN_DWELL']
        filtered_df = filtered_df[selected_columns]

        if os.path.isfile(ft_data_path):
            header_option = False
        else:
            header_option = True

        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)

        download_count += 1

    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1
df = pd.read_csv(ft_data_path)

with gzip.open(ft_data_path + ".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)
# Clean Visits Data
import json

visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_month_visit_summary' + file_suffix + '.csv'
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_month_visits' + file_suffix + '.csv'
df = pd.read_csv(
    "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/parks_hospitals_pharmacies_ft_data20230101_20240131.csv.gz")
df2 = df[
    ['PLACEKEY', 'DATE_RANGE_START', 'RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'DISTANCE_FROM_HOME', 'MEDIAN_DWELL']]
with gzip.open(visits_file + ".gz", 'wt', encoding='utf-8') as gzipped_file:
    df2.to_csv(gzipped_file, index=False)
# temp = df.loc[0]
# data = json.loads(temp['VISITS_BY_DAY'])
# data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
# data['PLACEKEY'] = temp['PLACEKEY']
# data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])

header_option_visit_summary_file = True

with tqdm(total=len(df)) as pbar:
    for index in range(0, len(df)):
        try:
            temp = df.loc[index]

            data = json.loads(temp['VISITS_BY_DAY'])
            data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
            data['PLACEKEY'] = temp['PLACEKEY']
            data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])

            if os.path.isfile(visit_summary_file):
                header_option_visit_summary_file = False

            data.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file, sep="|")
        except:
            pass

        pbar.update(1)
df = pd.read_csv(visit_summary_file, sep="|")
df.reset_index()
with gzip.open(visit_summary_file + ".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)